In [1]:
#pip install imblearn


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
import joblib
from joblib import load

In [3]:

# %cd ..
# %cd data

In [4]:
df = pd.read_csv("cleaned_data.csv")

df.head()

,YEAR_OF_DISCHARGE,AGE,GENDER,RACE,MARITAL_STATUS,EDUCATION,EMPLOYMENT_AT_ADMISSION,EMPLOYMENT_AT_DISCHARGE,LIVING_ARRANGEMENT_AT_ADMISSION,LIVING_ARRANGEMENT_AT_DISCHARGE,...,PRIMARY_SUBSTANCE_ABUSE,FREQUENCY_OF_USE,AGE_AT_FIRST_USE,ALCOHOL_OR_DRUG_ABUSE,DSM_DIAGNOSIS,PSYCHIATRIC_PROBLEM,HEALTH_INSURANCE,PRIMARY_PAYMENT_METHOD,FREQUENCY_OF_SELF_HELP_ATTENDANCE,STATE
0,2017,45-49,Female,White,"Divorced, widowed",Grade 12 (or GED),Full time,Full time,Independent living,Independent living,...,Methamphetamine/speed,No use in the past month,30 years and older,Other drugs only,Depressive disorders,No,None,Medicaid,8-30 times in the past month,Alaska
1,2017,35-39,Male,White,Never married,"4 years of college, university, BA/BS, some po...",Full time,Full time,Independent living,Independent living,...,Alcohol,Some use,18-20 years,Alcohol only,Alcohol abuse,No,"Medicare, other (e.g. TRICARE, CHAMPUS)","Private insurance (Blue Cross/Blue Shield, oth...",No attendance,Alaska
2,2017,30-34,Male,Two or more races,Never married,Grade 12 (or GED),Full time,Full time,Dependent living,Independent living,...,Alcohol,Some use,11 years and under,Alcohol and other drugs,Alcohol dependence,No,None,Medicaid,No attendance,Alaska
3,2017,25-29,Female,Alaskan Native,Now Married,Grades 9 to 11,Unemployed,Unemployed,Independent living,Independent living,...,Other drugs,Daily use,18-20 years,Alcohol and other drugs,Alcohol dependence,No,Medicaid,Medicaid,No attendance,Alaska
4,2017,25-29,Female,Alaskan Native,Now Married,Grades 9 to 11,Unemployed,Unemployed,Independent living,Independent living,...,Alcohol,Some use,12-14 years,Alcohol only,Alcohol dependence,Yes,Medicaid,Medicaid,No attendance,Alaska


In [5]:
print(df.columns)
print(df.shape)

Index(['YEAR_OF_DISCHARGE', 'AGE', 'GENDER', 'RACE', 'MARITAL_STATUS',
       'EDUCATION', 'EMPLOYMENT_AT_ADMISSION', 'EMPLOYMENT_AT_DISCHARGE',
       'LIVING_ARRANGEMENT_AT_ADMISSION', 'LIVING_ARRANGEMENT_AT_DISCHARGE',
       'ARRESTS_IN_30_DAYS_PRIOR_TO_ADMISSION',
       'ARRESTS_IN_30_DAYS_PRIOR_TO_DISCHARGE', 'SERVICES_AT_ADMISSION',
       'SERVICES_AT_DISCHARGE', 'REASON_FOR_DISCHARGE', 'LENGTH_OF_STAY',
       'PRIMARY_SOURCE_OF_REFERRAL', 'PRIOR_TREATMENT_EPISODES',
       'PRIMARY_SUBSTANCE_ABUSE', 'FREQUENCY_OF_USE', 'AGE_AT_FIRST_USE',
       'ALCOHOL_OR_DRUG_ABUSE', 'DSM_DIAGNOSIS', 'PSYCHIATRIC_PROBLEM',
       'HEALTH_INSURANCE', 'PRIMARY_PAYMENT_METHOD',
       'FREQUENCY_OF_SELF_HELP_ATTENDANCE', 'STATE'],
      dtype='object')
(1035841, 28)


## Feature Engineering

In [6]:
print(df['REASON_FOR_DISCHARGE'].value_counts(), '\n')
print(df['SERVICES_AT_DISCHARGE'].value_counts(), '\n')
print(df['PRIOR_TREATMENT_EPISODES'].value_counts())

Treatment completed                                     382503
Transferred to another treatment program or facility    345744
Dropped out of treatment                                180620
Terminated by facility                                   57002
Other                                                    52134
Incarcerated                                             16166
Death                                                     1672
Name: REASON_FOR_DISCHARGE, dtype: int64 

Ambulatory, non-intensive outpatient                465902
Ambulatory, intensive outpatient                    209535
Rehab/residential, short term (30 days or fewer)    139983
Detox, 24-hour, free-standing residential           131314
Rehab/residential, long term (more than 30 days)     57799
Ambulatory, detoxification                           18940
Detox, 24-hour, hospital inpatient                   11201
Rehab/residential, hospital (non-detox)               1167
Name: SERVICES_AT_DISCHARGE, dtype: int64 



In [7]:
df = df[df['REASON_FOR_DISCHARGE'] != 'Transferred to another treatment program or facility']

In [8]:
# Create target variable. If the patient completed treatment and had no prior treatment episodes, they are considered a success. Otherwise, they are considered a failure.
df['SUCCESSFUL_TREATMENT'] = df.apply(lambda row: 1 if row['REASON_FOR_DISCHARGE'] == 'Treatment completed' and row['PRIOR_TREATMENT_EPISODES'] == "No prior treatment episode" else 0, axis=1)

print(df['SUCCESSFUL_TREATMENT'].value_counts())

0    573270
1    116827
Name: SUCCESSFUL_TREATMENT, dtype: int64


## Modeling

### Initial evaluation/model baselines

In [22]:
df_one_hot = pd.get_dummies(df)
df_one_hot.shape

# for col in df_one_hot.columns:
#     print(col)

(690097, 224)

In [10]:
target = df['SUCCESSFUL_TREATMENT']
features = df.drop(['REASON_FOR_DISCHARGE', 'PRIOR_TREATMENT_EPISODES', 'SUCCESSFUL_TREATMENT'], axis=1)
features_one_hot = pd.get_dummies(features)

X_train, X_test, y_train, y_test = train_test_split(features_one_hot, target, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(552077, 215) (138020, 215) (552077,) (138020,)


In [21]:
for col in X_train.columns:
    print(col)

YEAR_OF_DISCHARGE
AGE_12-14
AGE_15-17
AGE_18-20
AGE_21-24
AGE_25-29
AGE_30-34
AGE_35-39
AGE_40-44
AGE_45-49
AGE_50-54
AGE_55-64
AGE_65+
GENDER_Female
GENDER_Male
RACE_Alaskan Native
RACE_American Indian
RACE_Asian
RACE_Asian or Pacific Islander
RACE_Black or African American
RACE_Native Hawaiian or Other Pacific Islander
RACE_Other single race
RACE_Two or more races
RACE_White
MARITAL_STATUS_Divorced, widowed
MARITAL_STATUS_Never married
MARITAL_STATUS_Now Married
MARITAL_STATUS_Separated
EDUCATION_1-3 years of college, university, or vocational school
EDUCATION_4 years of college, university, BA/BS, some postgraduate study, or more
EDUCATION_Grade 12 (or GED)
EDUCATION_Grades 9 to 11
EDUCATION_Less than one school grade, no schooling, nursery school, or kindergarten to Grade 8
EMPLOYMENT_AT_ADMISSION_Full time
EMPLOYMENT_AT_ADMISSION_Not in labor force
EMPLOYMENT_AT_ADMISSION_Part time
EMPLOYMENT_AT_ADMISSION_Unemployed
EMPLOYMENT_AT_DISCHARGE_Full time
EMPLOYMENT_AT_DISCHARGE_Not in 

In [11]:
# models_to_train = {
#     "naive_bayes": GaussianNB(),
#     "logistic_regression": LogisticRegression(max_iter=1000),
#     "random_forest": RandomForestClassifier()
# }

# for model_name, model in models_to_train.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     print(f"Accuracy score for {model_name}: {accuracy_score(y_test, y_pred)}")
#     print(f"f1 score for {model_name}: {f1_score(y_test, y_pred)}", '\n')

## Balance the data 

In [12]:
ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

print(y_train_res.value_counts())

0    458567
1    458567
Name: SUCCESSFUL_TREATMENT, dtype: int64


In [13]:
# for model_name, model in models_to_train.items():
#     model.fit(X_train_res, y_train_res)
#     y_pred = model.predict(X_test)
#     print(f"Accuracy score for {model_name}: {accuracy_score(y_test, y_pred)}")
#     print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}\n")

## Tuning

In [14]:
# # Define the parameter grid
# param_grid = {
#     'n_estimators': [250, 500], #100, 200, 400, 
#     'max_depth': [5,10], #5, 15, None
#     'min_samples_split': [2, 5], #2,10
#     'min_samples_leaf': [2,4] #1,4
# }

# # Define model
# rf = RandomForestClassifier()

# # Grid
# grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3)

# # Fit the grid search to the data
# grid_search.fit(X_train_res, y_train_res)

# # Print the best parameters
# print("Best parameters: ", grid_search.best_params_)

# # Save the model
# joblib.dump(grid_search.best_estimator_, 'best_random_forest.pkl')

In [15]:
%cd ..
%cd MADS-CAPSTONE

c:\Users\ericm\OneDrive\Documents\MADS\Capstone
c:\Users\ericm\OneDrive\Documents\MADS\Capstone\MADS-CAPSTONE


In [16]:
with open('best_random_forest.pkl', 'rb') as file:
    model = load(file)

y_pred = model.predict(X_test)
print(f"Accuracy score for random forest: {accuracy_score(y_test, y_pred)}")

C:\Users\ericm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ericm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Accuracy score for random forest: 0.6737864077669903
